In [1]:
import pandas as pd
from SPARQLWrapper import SPARQLWrapper, JSON
import sys
import numpy as np

In [2]:
dbpedia_endpoint = "https://dbpedia.org/sparql"
sparql = SPARQLWrapper(dbpedia_endpoint, agent='sparqlwrapper 1.8.6.dev0 (rdflib.github.io/sparqlwrapper)')

In [3]:
class_qry1 = """PREFIX wikiID: <http://www.wikidata.org/entity/>
select distinct ?ent ?type where { 
?ent owl:sameAs wikiID:"""
#Q76 to come here in between
class_qry2 = """ . ?ent rdf:type ?type 
FILTER( regex( ?type, "http://dbpedia.org/ontology/"))
}"""

In [4]:
#function to fetch results of search term
def fetch_ent_typ(qry):
       dbpedia_catg = ''
       lst_dbpd_typs = []
       try:
          sparql.setQuery(qry)
          sparql.setReturnFormat(JSON)
          results = sparql.query().convert()    #results['boolean']
          #print(results)
          if not len(results['results']['bindings']) == 0:
             for item in results['results']['bindings']:                   
                try:
                   dbpedia_catg = item['type']['type']
                   #print(dbpedia_catg)
                   lst_dbpd_typs.append('dbo:'+str(item['type']['value'].split('/')[-1]))
                except Exception as ex:
                   print(ex)
                   pass
       except Exception as ex:
          print('exception came: ',ex)
          pass
       if dbpedia_catg == 'uri':
          dbpedia_catg = 'resource'
       return dbpedia_catg, lst_dbpd_typs

In [7]:
#for i in range(4):
i=3
if i == 3:
    inp_file = '../data_files/simpques_wiki_smart/outdata'+str(i+1)+'.json'
    out_file = '../data_files/simpques_wiki_smart/output_sq_res'+str(i+1)+'.json'
    df_sq = pd.read_json(inp_file)
    #print(len(df_sq))
    lst_id, lst_ques, lst_datatype, lst_res = [], [], [], []
    for i in range(len(df_sq)):
        lst_id.append(df_sq['simple_questions_wikidata'][i]['id'])
        lst_ques.append(df_sq['simple_questions_wikidata'][i]['question'])
        wiki_obj = df_sq['simple_questions_wikidata'][i]['object']
        qry = class_qry1 + wiki_obj + class_qry2
        #print(qry)
        dbpedia_catg, dbpedia_type = fetch_ent_typ(qry)  #should return a category and a list 
        #print(dbpedia_catg, dbpedia_type)
        lst_datatype.append(dbpedia_catg)
        lst_res.append(dbpedia_type)
    pdf_data_v = {'id': lst_id, 'question': lst_ques, 'category': lst_datatype, 'type': lst_res}
    new_df = pd.DataFrame(pdf_data_v, columns = ['id','question','category','type'])
    new_df.to_json(out_file, orient='records', indent=len(new_df.columns))

## Processing SmartTask datasets
DBpedia datasets

In [10]:
dbpd_train_df = pd.read_json('../data_files/DBpedia/smarttask_dbpedia_train.json')
dbpd_test_df = pd.read_json('../data_files/DBpedia/smarttask_dbpedia_test_questions.json')
dbpd_test_gold_df = pd.read_json('../data_files/DBpedia/smarttask_dbpedia_test.json')
print(len(dbpd_train_df))
print(len(dbpd_test_df))
print(len(dbpd_test_gold_df))

17571
4381
4381


In [14]:
#merging train and test files
final_dbpd_df = pd.concat([dbpd_train_df, dbpd_test_df])
print(len(final_dbpd_df))
#fetching all records of web questions
webques_dbpd_inp_file = '../data_files/webques_wiki_smart/outdata_webques_dbepdia.json' 
wbqsp_dbpd_df = pd.read_json(webques_dbpd_inp_file)

21952


In [15]:
print('len of webqsp original: ',len(wbqsp_dbpd_df))
wbqsp_dbpd_df = wbqsp_dbpd_df.loc[wbqsp_dbpd_df['category']!='']
print('len of webqsp finetuned: ',len(wbqsp_dbpd_df))
final_dbpd_df = pd.concat([final_dbpd_df, wbqsp_dbpd_df])
for i in range(4):
    inp_file = '../data_files/simpques_wiki_smart/output_sq_res'+str(i+1)+'.json'  
    extra_dbpd_df = pd.read_json(inp_file)
    print('tot length of extra df: ',len(extra_dbpd_df))
    not_null_df = extra_dbpd_df.loc[extra_dbpd_df['category']!='']
    print('tot length of refined extra df: ',len(not_null_df))
    final_dbpd_df = pd.concat([final_dbpd_df, not_null_df])
print(len(final_dbpd_df))
#creating new id as old id no more holds good after merging all
final_dbpd_df['new_id'] = [x for x in range(len(final_dbpd_df))]
out_dbpd_file = '../data_files/simpques_wiki_smart/out_dbpd_train_test_merge.json'
final_dbpd_df.to_json(out_dbpd_file, orient='records', indent=len(final_dbpd_df.columns))
final_dbpd_df.tail(6)

len of webqsp original:  4131
len of webqsp finetuned:  4131
tot length of extra df:  5622
tot length of refined extra df:  5031
tot length of extra df:  6000
tot length of refined extra df:  5137
tot length of extra df:  13482
tot length of refined extra df:  11945
tot length of extra df:  2821
tot length of refined extra df:  2518
50714


,id,question,category,type,new_id
2813,2814,what kinds of music is played by season's end,resource,"[dbo:Genre, dbo:TopicalConcept, dbo:MusicGenre]",50708
2814,2815,which asteroid group is 6753 fursenko a member...,resource,[dbo:Album],50709
2815,2816,What language is azhakiya ravanan filmed in?,resource,[dbo:Language],50710
2816,2817,who developed outnumbered!,resource,"[dbo:Company, dbo:Organisation, dbo:Agent]",50711
2818,2819,which position did herby fortunat play in foot...,resource,[dbo:Person],50712
2820,2821,who is a person that was born in sao paulo,resource,"[dbo:Person, dbo:Artist, dbo:MusicalArtist, db...",50713


In [16]:
#renaming ids and dropping previous ids
final_dbpd_df.drop("id", axis=1, inplace=True)
final_dbpd_df.rename(columns={'new_id':'id'}, inplace=True)
final_dbpd_df = final_dbpd_df[['id','question','category','type']]

In [17]:
#splitting in train and test files
msk = np.random.rand(len(final_dbpd_df)) < 0.8
dbpd_train_df = final_dbpd_df[msk]
print(len(dbpd_train_df))
dbpd_test_gold_df = final_dbpd_df[~msk]
print(len(dbpd_test_gold_df))
out_dbpd_train_file = '../data_files/DBpedia/out_dbpd_train.json'
dbpd_train_df.to_json(out_dbpd_train_file, orient='records', indent=len(dbpd_train_df.columns))
out_dbpd_test_gold_file = '../data_files/DBpedia/out_dbpd_test_gold.json'
dbpd_test_gold_df.to_json(out_dbpd_test_gold_file, orient='records', indent=len(dbpd_test_gold_df.columns))
dbpd_test_df = dbpd_test_gold_df[['id','question']]
out_dbpd_test_file = '../data_files/DBpedia/out_dbpd_test.json'
dbpd_test_df.to_json(out_dbpd_test_file, orient='records', indent=len(dbpd_test_df.columns))

40621
10093


## Processing SmartTask datasets
Wikidata datasets

In [33]:
wikidata_endpoint = "https://query.wikidata.org/sparql"
sparql_wiki = SPARQLWrapper(wikidata_endpoint, agent='sparqlwrapper 1.8.6.dev0 (rdflib.github.io/sparqlwrapper)')

In [18]:
wiki_train_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_train.json')
wiki_test_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_test_gold.json')
wiki_test_gold_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_test.json')
print(len(wiki_train_df))
print(len(wiki_test_df))
print(len(wiki_test_gold_df))

18251
4571
4571


In [21]:
#merging train and test files
final_wiki_df = pd.concat([wiki_train_df, wiki_test_df])
print(len(final_wiki_df))

22822


In [22]:
qry_pt1 = 'select distinct ?label where { wd:'
qry_pt2 = ' rdfs:label ?label . FILTER (langMatches( lang(?label), "EN" ) ) } LIMIT 1'

In [37]:
#function to fetch answer from wikidata endpoint
def fetch_label(qry):
    res = ''
    sparql.setQuery(qry)
    sparql.setReturnFormat(JSON)
    results = sparql_wiki.query().convert() 
    for item in results['results']['bindings']: 
        res = item['label']['value'].lower().strip()
        print(res)
    return res

In [36]:
#fetching all records of web questions and simp ques
def concat_df(folder_name,num_files,file_name,subject,final_wiki_df):
    for j in range(num_files):
        webques_wiki_inp_file = folder_name+file_name+str(j+1)+'.json' 
        webques_wiki_out_file = folder_name+'lblOutput_'+file_name+str(j+1)+'.json' 
        wbqsp_wiki_df = pd.read_json(webques_wiki_inp_file)
        #print('len of webqsp original: ',len(wbqsp_wiki_df))
        #wbqsp_wiki_df = wbqsp_wiki_df.loc[wbqsp_wiki_df['category']!='']
        #print('len of webqsp finetuned: ',len(wbqsp_wiki_df))
        lst_id, lst_ques, lst_datatype, lst_res = [], [], [], []
        wiki_catg = ''
        for i in range(len(wbqsp_wiki_df)):
            lst_id.append(wbqsp_wiki_df[subject][i]['id'])
            lst_ques.append(wbqsp_wiki_df[subject][i]['question'])
            results = wbqsp_wiki_df[subject][i]['datatype']
            lst_tmp_lbls = []
            if not len(results['results']['bindings']) == 0:
                 for item in results['results']['bindings']:                   
                    try:
                       wiki_catg = item['type']['type']
                       #print(dbpedia_catg)
                       qry = qry_pt1 + str(item['type']['value'].split('/')[-1]) + qry_pt2
                       print(qry)
                       lst_tmp_lbls.append(fetch_label(qry))                       
                    except Exception as ex:
                       print(ex)
                       pass  
            if wiki_catg == 'uri':
               wiki_catg = 'resource'
            lst_datatype.append(wiki_catg)
            lst_res.append(lst_tmp_lbls)
            if i == 2:
               break
        pdf_data_v = {'id': lst_id, 'question': lst_ques, 'category': lst_datatype, 'type': lst_res}
        new_df = pd.DataFrame(pdf_data_v, columns = ['id','question','category','type'])
        new_df.to_json(webques_wiki_out_file, orient='records', indent=len(new_df.columns))
        final_wiki_df = pd.concat([final_wiki_df, new_df])
        return final_wiki_df

In [38]:
trial_df = final_wiki_df[0:3]
print(len(trial_df))
trial_df

3


,id,question,category,type
0,19719,What periodical literature does Delta Air Line...,resource,"[publication, recurring, intellectual work, te..."
1,15554,Who is the child of Ranavalona I's husband?,resource,"[person, omnivore, natural person]"
2,974,Is it true Jeff_Bridges occupation Lane Chandl...,boolean,[boolean]


In [ ]:
folder_name = '../data_files/webques_wiki_smart/'
file_name = 'outdata_wq'
subject = 'web_questions_wikidata'
new_final_wiki_df = concat_df(folder_name,3,file_name,subject,trial_df) #final_wiki_df
print(len(new_final_wiki_df))
new_final_wiki_df

select distinct ?label where { wd:Q95074 rdfs:label ?label . FILTER (langMatches( lang(?label), "EN" ) ) } LIMIT 1


In [ ]:
#creating new id as old id no more holds good after merging all
final_wiki_df['new_id'] = [x for x in range(len(final_wiki_df))]
out_wiki_file = '../data_files/simpques_wiki_smart/out_dbpd_train_test_merge.json'
final_wiki_df.to_json(out_wiki_file, orient='records', indent=len(final_wiki_df.columns))
final_wiki_df.tail(6)

In [50]:
wiki_train_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_train.json')
wiki_test_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_test.json')
wiki_test_gold_df = pd.read_json('../data_files/Wikidata/lcquad2_anstype_wikidata_test_gold.json')

In [51]:
print(len(wiki_train_df))
print(len(wiki_test_df))
print(len(wiki_test_gold_df))

18251
4571
4571


In [52]:
wiki_test_gold_df

,id,question,category,type
0,20258,Who is the country for head of state of Mahmo...,resource,"[country, disputed territory, state, Divided r..."
1,12761,Which female actress is the voice over on Sout...,resource,"[natural person, omnivore, person]"
2,4236,What is nominated for of Dolores del Río that ...,resource,"[award, class]"
3,27315,On Lake Winnipeg what is the lakes on river?,resource,[natural watercourse]
4,2623,What open cluster has the largest radius?,resource,[star cluster]
...,...,...,...,...
4566,24287,What are the gene which start with the letter t,resource,"[biological sequence, Nucleic acid sequence, b..."
4567,4667,For what work did Bernard DeVoto receive the P...,resource,"[written work, text, creative work]"
4568,16961,What is antonym of of spore print color of Ple...,resource,"[web-safe color, absorbed or reflected object ..."
4569,23859,Tell me mixture whose name has the word spirom...,resource,"[chemical substance, pharmaceutical preparatio..."


In [53]:
wiki_test_df

,id,question
0,20258,Who is the country for head of state of Mahmo...
1,12761,Which female actress is the voice over on Sout...
2,4236,What is nominated for of Dolores del Río that ...
3,27315,On Lake Winnipeg what is the lakes on river?
4,2623,What open cluster has the largest radius?
...,...,...
4566,24287,What are the gene which start with the letter t
4567,4667,For what work did Bernard DeVoto receive the P...
4568,16961,What is antonym of of spore print color of Ple...
4569,23859,Tell me mixture whose name has the word spirom...
